MTJ using 2D NEGF
This code used brute force NEGF to calucate current through a 2D layar
This code allow change of basis

In [1]:
#import numpy as np
import matplotlib.pyplot as plt
import time
import numpy as cp

'''Declearning Constand and Inputs =================='''
t1= time.time()
# Constants (all MKS, except energy which is in eV)
hbar = 1.06e-34
q = 1.6e-19
IE = (q*q) / (2 * cp.pi * hbar)
Ef = 2.25
kT = 0.025
Ecc = 3.02

#Effective mass in lattice for insulator and ferromagnet
mc = 0.18*9.1e-31
mf = 0.8*9.1e-31

# Inpupt a =size of lattice
a = 0.25e-10

#t0, tm are values of t in channel and magnet repectivily (NOTE:for now we have asumme energy Ec is same in both)
t0 = (hbar**2) / (2 * mc * (a**2) * q)
tm = (hbar**2) / (2 * mf * (a**2) * q)

#Lattice  point in Transverse Direction
Ds = 1 #Size of domain
Nd = 500 #No of domains
Np = Ds*Nd #Number of atom in transverse direction

#Lattic point in Source, channel and Drain
NS = 2
NC = 15
ND = 2
No = NS + NC + ND



Array "delt" represent angle of Non-Uniform Layer Spacially

In [2]:
'''These Hamaltonian are in transverse  direction and intially, Effective mass of ''' 
T_trans = 2*t0*cp.diag(cp.ones(2*Np))-t0*cp.diag(cp.ones(2*Np-2),2)-t0*cp.diag(cp.ones(2*Np-2),-2)

#Magnetic texture for right contact
delt = cp.pi*(cp.ones(Nd))
#delt = (cp.pi)*cp.zeros(Nd)
delt = cp.pi*(cp.linspace(0,1,Nd))
delta = (2.15)/2#Stoner model's spliting

#TR hamaltonain crossponding to spin interaction for right contact
Tdr = cp.zeros((2*Np,2*Np))

for j in range(Np):
    i = j//Ds
    nz = cp.cos(delt[i])
    nx = cp.sin(delt[i])
    Tdr[2*j:2*j+2,2*j:2*j+2] = delta*(cp.array([[nz,nx],[nx,-nz]]) +cp.eye(2))


TR= (tm/t0)*(T_trans) +Tdr

#Magnetic texture for right contact
delt = cp.zeros(Nd)
#delt = (cp.pi)*cp.ones(Nd)
delta = (2.15)/2#Stoner model's spliting

#TR hamaltonain crossponding to spin interaction for right contact
Tdl = cp.zeros((2*Np,2*Np))

for j in range(Np):
    i = j//Ds
    nz = cp.cos(delt[i])
    nx = cp.sin(delt[i])
    Tdl[2*j:2*j+2,2*j:2*j+2] = delta*(cp.array([[nz,nx],[nx,-nz]]) +cp.eye(2))


TL= (tm/t0)*(T_trans) +Tdl
#Calculating eigenvalues and eigenvectors of left and write contact
Eig_R,Unit_R = cp.linalg.eigh(TR)
Eig_L,Unit_L = cp.linalg.eigh(TL)


'''Hamaltonain in direction of current'''
diag0 = cp.zeros(No)

for i in range((NS-1)):
    diag0[i] = 2*tm
    diag0[(No-i)-1] = 2*tm

for i in range((NC)):
    diag0[(NS+i)] = 2*t0

#Terms to account for variable 
diag0[NS-1] = tm+t0
diag0[(No-NS)] = tm+t0



#Off diagonal terms
diag2 = cp.zeros((No-1))
diag2[:(NS-1)] = tm
diag2[(NS+NC):] = tm
diag2[(NS-1):(NS+NC)] = t0


Tlong = (cp.diag(diag0) - (cp.diag(diag2, k=1)) - (cp.diag(diag2, k=-1)))
    

#modifyin H to account for tight biding in x
T_trans=2*t0*cp.eye(2*Np)+T_trans
TL=2*tm*cp.eye(2*Np)+TL
TR=2*tm*cp.eye(2*Np)+TR

In [ ]:
#Defining potentail profile
UB = cp.zeros(No)
UB[NS:NS+NC]=Ecc

# Bias
V_range = cp.linspace(0, 0, 1) # Range of bias voltages to calculate
V_range = [0.05]
I_array = [] # Array to store the currents for each bias voltage

NE=40
Iee=cp.zeros(NE)

for V in V_range:
    mu1 = Ef + (V/2)
    mu2 = Ef - (V/2)
    #Applying voltage and potentail texture
    UX = V* cp.concatenate((0.5 * cp.ones(2*NS), cp.linspace(0.5, -0.5, 2*NC), -0.5 * cp.ones(2*ND)))
    U1 = UX[0:2*No:2]
    U1=U1+UB
    # Transmission
    TM = cp.zeros(NE)
    # Energy grid for Green's function method
    E = cp.linspace(mu2-10*kT, mu1+10*kT, NE)
    zplus = 1j * 1e-12
    dE = E[1] - E[0]
    f1 = 1 / (1 + cp.exp((E - mu1) / kT))
    f2 = 1 / (1 + cp.exp((E - mu2) / kT))
    Is=cp.zeros(NE)
    I=0
    #E= E[99]*cp.ones(100)
    for k in range(NE):
      '''Getting surface green function for both contacts'''
      #Right contact
      #Adjust eignvalues according to mass of contact
      ck = 1 - ((E[k] + zplus-U1[0] -Eig_L) / (2*tm))
      ka = cp.arccos(ck)
      Green = -tm * cp.exp(1j*ka)
      sigm1 = Unit_L@cp.diag(Green)@Unit_L.T
      
      #Left contact
      ck = 1 - ((E[k] + zplus -U1[No-1] -Eig_R)/(2*tm))
      ka = cp.arccos(ck)
      Green = -tm * cp.exp(1j*ka)
      sigm2 = Unit_R@cp.diag(Green)@Unit_R.T
      #Calcuating sigma
      [G11,GN1]=Inv_surf_D(T_trans,TL,TR,E[k],U1,0,sigm1,sigm2)
      [G11,GN2]=Inv_surf_D(T_trans,TL,TR,E[k],U1,1,sigm1,sigm2)
      [G11,G12]=Inv_surf(T_trans,TL,TR,E[k],U1,2,sigm1,sigm2)

      #Calculating gama1 and gama2
      sigm1 = -2*cp.imag(sigm1)
      sigm2 = -2*cp.imag(sigm2)

      #Calculating A12 and A22
      sigm1 = G11@sigm1@(cp.conj(G12).T)
      sigm2 = GN1@sigm2@(cp.conj(GN2).T)
      Gn = sigm1*f1[k]+sigm2*f2[k]
      Iop = -2*tm*(cp.imag(Gn))
      Iee[k]=cp.real(cp.trace(Iop))/(f1[k]-f2[k])
      Is[k]=IE*cp.real(cp.trace(Iop))
    
    I=cp.trapz(Is,dx=dE)
  
print(I)

In [ ]:
plt.plot(Iee)

In [3]:
#Contain function that can calcuate specific section of inverse

def Inv_surf(T_trans,T_L,T_R,E,V,p,sigm1,sigm2):
  '''This matrix calcuate the first row of a Green function, T is the traverse hamatonain(without effect of magnetization and effective mass is taken that of a channel), E is Energy, V is votlage profile p is element of raw you want to calcuate
  Altough Simg1 and simg2 are sigma's of respective matrix but modification due to hamatonain due to magnetic texture can be added with these'''
  
  #B is will (1,1) matrix and Xm will be (1,p) matrix 
  B=cp.zeros((2*Np,2*Np),dtype=complex)
  Xm=cp.zeros((2*Np,2*Np),dtype=complex)
  #Adding effect of tight binding in long direction
  Ez = (E+zplus)*cp.eye(2*Np)
  Mcon = tm/t0 #To convert system with effective mass of mc to mf
  Mjuc = (tm+t0)/(2*t0) #To convert system with effective mass of mc to mf+mc/2
  #Calculating surface of first 2x2 matrix
  A =Ez-T_R - sigm2 -V[No-1]*cp.eye(2*Np)
  B = cp.linalg.inv(A)
  if p==No:
      Xm = B
  for i in range(No-2):
    if i==0:
      A =Ez-((T_trans+T_R-Tdr)/2) -V[No-2-i]*cp.eye(2*Np)
      B = A - (tm**2)*B
    elif i == No-3:
      A =Ez-((T_trans+T_L-Tdl)/2) - V[No-2-i]*cp.eye(2*Np)
      B = A - (t0**2)*B
    else:
      A =Ez-(T_trans) - V[No-2-i]*cp.eye(2*Np)
      B = A - (t0**2)*B
    B = cp.linalg.inv(B)
    if i==No-1-p:
      Xm = B
    if i>=No-p:
      if i==0:
        Xm= -1*(tm)*Xm@B
      else:
        Xm = -1*(t0)*Xm@B

  A =Ez-T_L - sigm1 - V[0]*cp.eye(2*Np)
  B = (A) - (tm**2)*B
  B = cp.linalg.inv(B)
  Xm = -1*(tm)*Xm@B
  return [B,Xm]

  '''---------------------------------------------------------------------------'''

def Inv_surf_D(T_trans,T_L,T_R,E,V,p,sigm1,sigm2):
  '''This matrix calcuate the first row of a Green function, T is the traverse hamatonain(without effect of magnetization and effective mass is taken that of a channel), E is Energy, V is votlage profile p is element of raw you want to calcuate
  Altough Simg1 and simg2 are sigma's of respective matrix but modification due to hamatonain due to magnetic texture can be added with these'''
  
  #B is will (1,1) matrix and Xm will be (1,p) matrix 
  B=cp.zeros((2*Np,2*Np),dtype=complex)
  Xm=cp.zeros((2*Np,2*Np),dtype=complex)
  #Adding effect of tight binding in long direction
  Ez = (E+zplus)*cp.eye(2*Np)
  Mcon = tm/t0 #To convert system with effective mass of mc to mf
  Mjuc = (tm+t0)/(2*t0) #To convert system with effective mass of mc to mf+mc/2
  #Calculating surface of first 2x2 matrix
  A =Ez-TL - sigm1 -V[0]*cp.eye(2*Np)
  B = cp.linalg.inv(A)
  if p==0:
      Xm = B
  for i in range(1,No-1):
    if i==1:
      A =Ez-((T_trans)*(Mjuc)) -V[i]*cp.eye(2*Np)
      B = A - (tm**2)*B
    elif i == No-2:
      A =Ez-((T_trans)*(Mjuc)) - V[i]*cp.eye(2*Np)
      B = A - (t0**2)*B
    else:
      A =Ez-(T_trans) - V[i]*cp.eye(2*Np)
      B = A - (t0**2)*B
    B = cp.linalg.inv(B)

    if i==p:
      Xm=B
    if i>=p+1:
      if i==1:
        Xm=-tm*Xm@B
      else:  
        Xm=-t0*Xm@B

  A =Ez-TR - sigm2 - V[No-1]*cp.eye(2*Np)
  B = (A) - (tm**2)*B
  B = cp.linalg.inv(B)
  Xm = -1*(tm)*Xm@B
  return [B,Xm]


